In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import time
import klib

In [26]:
# load processed dataset
data_bin_hot2 = pd.read_csv('X-IIoTID dataset_processed.csv')

### LSTM Model for Binary Classification

In [16]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn import metrics

# Drop last 18 columns for X and remove class3
X = (data_bin_hot2.iloc[:,:-18]).drop(columns = ['class3'], axis=1)
Y = data_bin_hot2['class3']


from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)

# convert Y to int
Y_train = Y_train.astype(int)
Y_test = Y_test.astype(int)

# reshape the data for LSTM
X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

# convert all to numpy arrays
X_train = np.asarray(X_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)


# define the model
model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(64))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
start = time.time()
# train the model
history = model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_test, Y_test), shuffle=True)
end = time.time()
print("Time taken to train the model: ", end - start)

Epoch 1/10
14901/14901 [==============================] - 34s 2ms/step - loss: 0.0734 - accuracy: 0.9765 - val_loss: 0.0540 - val_accuracy: 0.9809
Epoch 2/10
14901/14901 [==============================] - 33s 2ms/step - loss: 0.0558 - accuracy: 0.9802 - val_loss: 0.0474 - val_accuracy: 0.9814
Epoch 3/10
14901/14901 [==============================] - 33s 2ms/step - loss: 0.0500 - accuracy: 0.9809 - val_loss: 0.0456 - val_accuracy: 0.9823
Epoch 4/10
14901/14901 [==============================] - 30s 2ms/step - loss: 0.0471 - accuracy: 0.9815 - val_loss: 0.0428 - val_accuracy: 0.9834
Epoch 5/10
14901/14901 [==============================] - 30s 2ms/step - loss: 0.0452 - accuracy: 0.9822 - val_loss: 0.0411 - val_accuracy: 0.9830
Epoch 6/10
14901/14901 [==============================] - 30s 2ms/step - loss: 0.0442 - accuracy: 0.9826 - val_loss: 0.0402 - val_accuracy: 0.9839
Epoch 7/10
14901/14901 [==============================] - 30s 2ms/step - loss: 0.0431 - accuracy: 0.9828 - val_loss: 0

In [18]:
start = time.time()
y_pred = model.predict(X_test)
end = time.time()
y_pred = (y_pred > 0.5)
print('Classification Report')
print(metrics.classification_report(Y_test, y_pred))
print('Accuracy:')
print(metrics.accuracy_score(Y_test, y_pred))
print("Time taken to predict: ", end - start)

3726/3726 [==============================] - 2s 614us/step
Classification Report
              precision    recall  f1-score   support

           0       0.99      0.98      0.98     51897
           1       0.98      0.99      0.99     67311

    accuracy                           0.99    119208
   macro avg       0.99      0.98      0.99    119208
weighted avg       0.99      0.99      0.99    119208

Accuracy:
0.9854036641836118
Time taken to predict:  2.838178873062134


In [30]:
# save model
model.save('X-IIoTID_lstm_bin.h5')

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
